In [2]:
#Libraries
import pandas as pd
import logging
import re
import ast
import os
import wget
from datetime import datetime
import sqlite3
import pymysql

In [1]:
# #Input the data having Download JSON
# data = pd.read_excel('C:/Users/GOWRICH1/OneDrive - Novartis Pharma AG/Atezolizumab/')
# data.shape

(37, 14)

In [3]:
#setting up connection
con = pymysql.connect(host='rds-hawkaif1dev01.aws.novartis.net',
                      user='hawkaif1dev01',
                      password='M4F5jPThk1o1PSMn',
                      database='hawkaif1dev01',
                      port=3306,
                     charset='utf8')

In [4]:
#Tables in Database
cursor = con.cursor() 
cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = 'hawkaif1dev01'")

# Read and print tables
for table in [tables[0] for tables in cursor.fetchall()]:
    print(table)

EMPLOYEE
Insights_molecule_data
NKC_DH_download_matrix
Outcome_matrix
POS_data
Synonyms_matrix
book_details
hawkai_molecules


In [5]:
# Data input from SQL Table
sql_query = pd.read_sql_query ('''
                               SELECT
                               *
                               FROM NKC_DH_download_matrix
                               ''', con) #Input the table which need to be read

data = pd.DataFrame(sql_query)
data

molecule name                                     (and) keywords  \
0    Atezolizumab  non small cell lung cancer , breast cancer , m...   
1   Pembrolizumab  Melanoma, Non-Small Cell Lung Cancer (NSCLC), ...   
2       Nivolumab  \nAdvanced Esophageal Adenocarcinoma\nAdvanced...   
3      Durvalumab  \nUnresectable Stage III Non-small Cell Lung C...   
4     ipilimumab   \nCutaneous Melanoma\nHepatocellular Carcinoma...   
5    Tislelizumab                                               None   
6      Cemiplimab  \nLocally Advanced Basal Cell Carcinoma\nLocal...   
7        Avelumab  \nAdvanced Renal Cell Carcinoma (aRCC)\nMetast...   
8   Spartalizumab                                               None   
9      Sintilimab                                               None   
10   Tremelimumab        \nCP 675,206\nCP-675\nCP-675,206\nCP-675206   
11    Quavonlimab                                               None   
12    Daratumumab                     multiple myeloma , transplant    
13     Isatuximab                                 multiple myeloma     
14    Ustekinumab  \nSevere Plaque psoriasis\nUlcerative Colitis,...   
15    Secukinumab  \nEnthesitis Related Arthritis (ERA)\nJuvenile...   
16     Ixekizumab  Moderate to Severe Plaque Psoriasis\nActive An...   
17    Mepolizumab  Chronic Rhinosinusitis Phenotype With Nasal Po...   
18     Guselkumab              Plaque psoriasis, psoriatic arthritis   
19    Tocilizumab  MRA\nMSB-11456\nMSB11456\nR-1569\nRG-1569\nRHP...   
20   Benralizumab                         Severe Eosinophilic Asthma   
21      Sarilumab  Moderate, active Rheumatoid arthritis\nSevere,...   
22     Brodalumab  Severe Plaque psoriasis\nModerate Plaque psori...   
23   Risankizumab                                   Plaque psoriasis   
24     Reslizumab                         Severe Eosinophilic Asthma   
25  Tildrakizumab                Moderate to Severe Plaque Psoriasis   
26    Canakinumab  \nAdult Onset Still's Disease\nCryopyrin-assoc...   
27   Tralokinumab               Moderate to Severe Atopic Dermatitis   
28     Daclizumab                 Relapsing Multiple Sclerosis (RMS)   
29      Rituximab  \nActive, Moderate to Severe Rheumatoid Arthri...   
30    Natalizumab  Anti-alpha4 integrin\nAnti-VLA4\nRelapsing Rem...   
31    Ocrelizumab  relapsing remitting multiple sclerosis , prima...   
32     Ofatumumab  relapsing remitting multiple sclerosis , prima...   
33     Emicizumab                    Bleeding caused by Hemophilia A   
34    Trastuzumab  Breast Cancer\nEarly Breast Cancer\nInflammato...   
35     Pertuzumab  Early Breast Cancer\nInflammatory Breast Cance...   
36      Dupilumab  Chronic Rhinosinusitis Phenotype With Nasal Po...   

                                        (or) keywords key metrics  \
0                 Tecentriq, MPDL3280A, and anti-PDL1        None   
1                Keytruda, MK-3475, and Lambrolizumab        None   
2   BMS 936558\nBMS-936558\nGTPL7335\nMDX 1106\nMD...        None   
3             Imfinzi\nMEDI 4736\nMEDI-4736\nMEDI4736        None   
4   Yervoy\nBMS-734016\nMDX-010\nMDX-101\nMDX-CTLA...        None   
5                                            BGB-A317        None   
6       Libtayo\nCemiplimab-rwlc\nREGN-2810\nREGN2810        None   
7                                            Bavencio        None   
8                                              PDR001        None   
9                                   \nIBI-308\nIBI308        None   
10  Ticilimumab\nCP 675,206\nCP-675\nCP-675,206\nC...        None   
11                     Mk-1308A component quavonlimab        None   
12  anti-CD38 monoclonal antibody, Darzalex, and h...        None   
13              Sarclisa, SAR-650984, isatuximab-irfc        None   
14  Stelera\nCNTO 1275\nCNTO-1275\nCNTO1275\nL04AC...        None   
15                 Cosentyx\nAIN-457\nAIN457\nAIN457A        None   
16           Taltz\nLY 2439821\nLY-2439821\nLY2439821        None   
17                       

In [2]:
data.head()

molecule name                                     (and) keywords  \
0   Atezolizumab  non small cell lung cancer , breast cancer , m...   
1  Pembrolizumab  Melanoma, Non-Small Cell Lung Cancer (NSCLC), ...   
2      Nivolumab  \nAdvanced Esophageal Adenocarcinoma\nAdvanced...   
3     Durvalumab  \nUnresectable Stage III Non-small Cell Lung C...   
4    ipilimumab   \nCutaneous Melanoma\nHepatocellular Carcinoma...   

                                       (or) keywords  key metrics  \
0                Tecentriq, MPDL3280A, and anti-PDL1          NaN   
1               Keytruda, MK-3475, and Lambrolizumab          NaN   
2  BMS 936558\nBMS-936558\nGTPL7335\nMDX 1106\nMD...          NaN   
3            Imfinzi\nMEDI 4736\nMEDI-4736\nMEDI4736          NaN   
4  Yervoy\nBMS-734016\nMDX-010\nMDX-101\nMDX-CTLA...          NaN   

   example key metric patterns  key comparators  additional comments  \
0                          NaN              NaN                  NaN   
1                          NaN              NaN                  NaN   
2                          NaN              NaN                  NaN   
3                          NaN              NaN                  NaN   
4                          NaN              NaN                  NaN   

                                           NKC query  \
0             Atezolizumab OR Tecentriq OR MPDL3280A   
1  Pembrolizumab OR Keytruda OR MK-3475 OR Lambro...   
2  Nivolumab OR BMS 936558 OR BMS-936558 OR GTPL7...   
3     Durvalumab OR Imfinzi OR MEDI-4736 OR MEDI4736   
4  ipilimumab OR Yervoy OR BMS-734016 OR MDX-010 ...   

                                     mcq id type of harvest year range   \
0                             SAVIOEL2.1226         ongoing   2000-2022   
1                             SAVIOEL2.1227         ongoing   2000-2022   
2  SAVIOEL2.1228 - updated to SAVIOEL2.1263         ongoing   2000-2022   
3                             SAVIOEL2.1229         ongoing   2000-2022   
4                             SAVIOEL2.1230         ongoing   2000-2022   

                 harvest_name  \
0   Atezolizumab_2022_ongoing   
1  Pembrolizumab_2022_ongoing   
2      Nivolumab_2022_ongoing   
3     Durvalumab_2022_ongoing   
4     ipilimumab_2022_ongoing   

                                        dowload json  \
0  on_message %s {"extract.outfile":["/data/nkc_r...   
1  on_message %s {"extract.outfile":["/data/nkc_r...   
2  on_message %s {"extract.outfile":["/data/nkc_r...   
3  on_message %s {"extract.outfile":["/data/nkc_r...   
4  on_message %s {"extract.outfile":["/data/nkc_r...   

                                        dowload path  
0  /Users/savioel2/Desktop/HawkAISandoz/2022_rele...  
1  /Users/savioel2/Desktop/HawkAISandoz/2022_rele...  
2  /Users/savioel2/Desktop/HawkAISandoz/2022_rele...  
3  /Users/savioel2/Desktop/HawkAISandoz/2022_rele...  
4  /Users/savioel2/Desktop/HawkAISandoz/2022_rele...

In [4]:
#Logging configuration
logging.basicConfig(filename="Downloads/hawkai_files/hawkai_log.txt",#specify the folder location where log file(replace with s3/dbfs data source)
                   filemode="a",#append mode
                   format="%(asctime)s %(levelname)s-%(message)s",#format of the log message
                   datefmt="%Y-%m-%d %H:%M:%S")

In [5]:
#Logging object
logger = logging.getLogger()
logger.setLevel(logging.DEBUG) # Sets the threshold of logger to DEBUG

In [6]:
#Function to Extract download links from download_json
def extract_download_links(json_data):    
    try:
        return ast.literal_eval(re.search('({.+})', json_data).group(0))['extract.download']
    except:
        return 'Data/Links Missing'

In [7]:
#Create a new column to append download links
data['download links']= data['dowload json'].apply(lambda x: extract_download_links(x))

In [32]:
data.head()

molecule name                                     (and) keywords  \
0   Atezolizumab  non small cell lung cancer , breast cancer , m...   
1  Pembrolizumab  Melanoma, Non-Small Cell Lung Cancer (NSCLC), ...   
2      Nivolumab  \nAdvanced Esophageal Adenocarcinoma\nAdvanced...   
3     Durvalumab  \nUnresectable Stage III Non-small Cell Lung C...   
4    ipilimumab   \nCutaneous Melanoma\nHepatocellular Carcinoma...   

                                       (or) keywords  key metrics  \
0                Tecentriq, MPDL3280A, and anti-PDL1          NaN   
1               Keytruda, MK-3475, and Lambrolizumab          NaN   
2  BMS 936558\nBMS-936558\nGTPL7335\nMDX 1106\nMD...          NaN   
3            Imfinzi\nMEDI 4736\nMEDI-4736\nMEDI4736          NaN   
4  Yervoy\nBMS-734016\nMDX-010\nMDX-101\nMDX-CTLA...          NaN   

   example key metric patterns  key comparators  additional comments  \
0                          NaN              NaN                  NaN   
1                          NaN              NaN                  NaN   
2                          NaN              NaN                  NaN   
3                          NaN              NaN                  NaN   
4                          NaN              NaN                  NaN   

                                           NKC query  \
0             Atezolizumab OR Tecentriq OR MPDL3280A   
1  Pembrolizumab OR Keytruda OR MK-3475 OR Lambro...   
2  Nivolumab OR BMS 936558 OR BMS-936558 OR GTPL7...   
3     Durvalumab OR Imfinzi OR MEDI-4736 OR MEDI4736   
4  ipilimumab OR Yervoy OR BMS-734016 OR MDX-010 ...   

                                     mcq id type of harvest year range   \
0                             SAVIOEL2.1226         ongoing   2000-2022   
1                             SAVIOEL2.1227         ongoing   2000-2022   
2  SAVIOEL2.1228 - updated to SAVIOEL2.1263         ongoing   2000-2022   
3                             SAVIOEL2.1229         ongoing   2000-2022   
4                             SAVIOEL2.1230         ongoing   2000-2022   

                 harvest_name  \
0   Atezolizumab_2022_ongoing   
1  Pembrolizumab_2022_ongoing   
2      Nivolumab_2022_ongoing   
3     Durvalumab_2022_ongoing   
4     ipilimumab_2022_ongoing   

                                        dowload json  \
0  on_message %s {"extract.outfile":["/data/nkc_r...   
1  on_message %s {"extract.outfile":["/data/nkc_r...   
2  on_message %s {"extract.outfile":["/data/nkc_r...   
3  on_message %s {"extract.outfile":["/data/nkc_r...   
4  on_message %s {"extract.outfile":["/data/nkc_r...   

                                        dowload path  \
0  /Users/savioel2/Desktop/HawkAISandoz/2022_rele...   
1  /Users/savioel2/Desktop/HawkAISandoz/2022_rele...   
2  /Users/savioel2/Desktop/HawkAISandoz/2022_rele...   
3  /Users/savioel2/Desktop/HawkAISandoz/2022_rele...   
4  /Users/savioel2/Desktop/HawkAISandoz/2022_rele...   

                                      download links  
0  [http://skc10.nibr.novartis.net/repository/ext...  
1  [http://skc10.nibr.novartis.net/repository/ext...  
2  [http://skc10.nibr.novartis.net/repository/ext...  
3  [http://skc10.nibr.novartis.net/repository/ext...  
4  [http://skc10.nibr.novartis.net/repository/ext...

In [9]:
#Function to create directory to each molecule name 
#(please replace below folder path with s3/dbfs data source where download files needs to stored)
def create_directory_with_molecule(molecule_name):
    directory = molecule_name
    parent_dir = "C:/Users/GOWRICH1/Downloads/hawkai_files/" #Input the folder path where data need to be downloaded
    path = os.path.join(parent_dir, directory)
    try:
        os.mkdir(path)
        logger.info('Folder created for %s'%directory)
        return path
    except:
        logger.warning('Folder name "%s" already present in Directory(%s) (or) Incorrect Folder Name format'%(directory,parent_dir))
        return path
    
#below code module can be used take a input from user whether to continue download or not,if the folder is already present.
        '''
        if (input('Do you wish to continue download? (yes/no) : ')!='yes'):
            return None
        else:
            return path
        '''

In [26]:
# Function to download zip file for download URL
def download_URL_file(download_url,target_path,molecule_name):
    logger.info('Download started for %s'%molecule_name)
    try:
        wget.download(download_url,out=target_path)
        logger.info('File Downloaded for %s and available in %s'%(molecule_name,target_path))
        dow_time=datetime.now()
        return ["Download Successful",dow_time]
    except:
        logger.error('Failed to Download %s file (URL - %s)'%(molecule_name,download_url))
        #dow_time=datetime.now()
        return ["Download Unsuccessful","NA"]

In [11]:
# Function to iterate through the each molecules download URL
def URL_data(molecule_name,download_links):
    if download_links!='Data/Links Missing':
        target_path = create_directory_with_molecule(molecule_name)
        if target_path!=None:
            for download_link in download_links:
                status = download_URL_file(download_link,target_path,molecule_name) 
            return status
        else:
            logger.error('Download Failed for %s'%(molecule_name))
            return ['Download Unsuccessful',"NA"]  
    else:
        logger.error('No links available, Download Failed for %s'%(molecule_name))
        return ['Download Unsuccessful',"NA"]

In [ ]:
# To intiate data download and updating download details to the dataframe
data[['Download Status','Download Time']] = pd.DataFrame(data.apply(lambda x: URL_data(x['molecule name'], x['download links']), axis=1).tolist(), index= data.index)

### Test Example

In [29]:
data1 = data[3:4]
data1

molecule name                                     (and) keywords  \
3    Durvalumab  \nUnresectable Stage III Non-small Cell Lung C...   

                             (or) keywords  key metrics  \
3  Imfinzi\nMEDI 4736\nMEDI-4736\nMEDI4736          NaN   

   example key metric patterns  key comparators  additional comments  \
3                          NaN              NaN                  NaN   

                                        NKC query         mcq id  \
3  Durvalumab OR Imfinzi OR MEDI-4736 OR MEDI4736  SAVIOEL2.1229   

  type of harvest year range              harvest_name  \
3         ongoing   2000-2022  Durvalumab_2022_ongoing   

                                        dowload json  \
3  on_message %s {"extract.outfile":["/data/nkc_r...   

                                        dowload path  \
3  /Users/savioel2/Desktop/HawkAISandoz/2022_rele...   

                                      download links  
3  [http://skc10.nibr.novartis.net/repository/ext...

In [30]:
# To intiate data download and updating download details to the dataframe
data1[['Download Status','Download Time']] = pd.DataFrame(data1.apply(lambda x: URL_data(x['molecule name'], x['download links']), axis=1).tolist(), index= data1.index)

100% [......................................................................] 352677257 / 352677257

In [31]:
data1

molecule name                                     (and) keywords  \
3    Durvalumab  \nUnresectable Stage III Non-small Cell Lung C...   

                             (or) keywords  key metrics  \
3  Imfinzi\nMEDI 4736\nMEDI-4736\nMEDI4736          NaN   

   example key metric patterns  key comparators  additional comments  \
3                          NaN              NaN                  NaN   

                                        NKC query         mcq id  \
3  Durvalumab OR Imfinzi OR MEDI-4736 OR MEDI4736  SAVIOEL2.1229   

  type of harvest year range              harvest_name  \
3         ongoing   2000-2022  Durvalumab_2022_ongoing   

                                        dowload json  \
3  on_message %s {"extract.outfile":["/data/nkc_r...   

                                        dowload path  \
3  /Users/savioel2/Desktop/HawkAISandoz/2022_rele...   

                                      download links      Download Status  \
3  [http://skc10.nibr.novartis.net/repository/ext...  Download Successful   

               Download Time  
3 2022-07-15 10:41:06.977129

In [6]:
from dateutil import parser

data = ['Peter drinks tea at september', 'My birthday is on january', "On Sat 11 July I'll be back home"]

def is_valid_date(date_str):
    try:
        parser.parse(date_str)
        return True
    except:
        return False

new_list = [' '.join([w for w in line.split() if not is_valid_date(w)]) for line in data]
print(new_list)

['Peter drinks tea at', 'My birthday is on', "On I'll be back home"]


In [134]:
def clean_comparator(comp):
    lii=[]
    for char in comp.split(","):    
        if all(digit not in char for digit in "1234567890"):
            month=[w for w in char.split() if not is_valid_date(w)]
            if(month)!=[]:
                lii.append(' '.join(month))
    return lii

In [136]:
clean_comparator('dimethyl fumarate,vumerity,dhodh inhibitor,amino acid oligomers,novantrone,ocrevus,lemtrada,tysabri,fumarate,copaxone,mitoxantrone')

['dimethyl fumarate',
 'vumerity',
 'dhodh inhibitor',
 'amino acid oligomers',
 'novantrone',
 'ocrevus',
 'lemtrada',
 'tysabri',
 'fumarate',
 'copaxone',
 'mitoxantrone']

In [132]:
def clean_intervention(intervention):
    return unicodedata.normalize("NFKD",intervention)

In [128]:
def get_dosage(intervention):
    weightpatterns = re.compile('(?<!\d|\.)\d+(?:\.\d+)?\s*?(?:mg|kg|ml|q\.s\.|ui|M|g|µg)(?!\w)')
    return list(set(weightpatterns.findall(intervention)))

In [133]:
get_dosage(clean_intervention('study design  study omb112855 was a single‐arm phase i study in patients with fludarabine‐refractory cll who received 12 infusions of ofatumumab over a six‐month period (300 mg at week 1 2000 mg weekly for seven doses then 2000 mg every four weeks for four doses).'))

['2000 mg', '300 mg']

In [151]:
def filter_percentage(val):
    if float(val.strip('%'))<=100:
        return val
    else:
        pass

In [157]:
filter_percentage('100%')

'100%'